<a href="https://colab.research.google.com/github/smbonilla/learningPyTorch/blob/main/imageGenerationUsingGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Image Generation using Generative Adversarial Networks

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
device

'cuda'

In [3]:
!pip install jovian --upgrade -q
import jovian
jovian.__version__

     |████████████████████████████████| 68 kB 6.1 MB/s 


'0.2.45'

In [4]:
jovian.utils.colab.set_colab_file_id('1Bz0oJP4zOfTu3Zd2QhFAvY0EXj_119E6')

## Approach towards GAN

Random input vector -> Generator Model -> Generated Example

Generated Example + Real Example -> Discriminator Model -> Binary Classification (real/fake) -> update discriminator model + generator model 

1. train the discriminator model to distinguish between really bad generator images and real images (binary classification model)
2. train the generator model for a few epochs to create better fakes 
3. repeat 1&2 over and over




## Use Anime Face Dataset

consists of 63,000 cropped anime faces - unsupervised and so these images do not have labels


### get dataset from kaggle

use opendatasets library to download kaggle dataset
1. sign in to kaggle and click on profile picture and select my account
2. scroll down to API and click create new API Token this will download a file kaggle.json 
3. when you run `opendatasets.download` you will be asked to enter your username & kaggle API which you get from previous file